In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import numpy as np
import os
import torch.nn.init as init
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from scipy.stats import spearmanr
import json


#FIRST VARIANT OF THE HUMAN VISUAL SYSTEM
class ReferenceVisualNetwork(nn.Module):
    """
    A simple feedforward CNN with two parallel pathways (Ventral and Dorsal)
    to serve as the baseline for the assignment.
    """
    def __init__(self):
        super(ReferenceVisualNetwork, self).__init__()
        
        # --- 1. Initial Feature Extraction (Shared V1/V2 - Early Visual Areas) ---
        # Input size for Fashion MNIST: (1, 28, 28)
        # These layers model the initial processing common to both pathways (V1, V2, Area hOc1, Area hOc2).
        self.v1_v2_conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1) # (1, 28, 28) -> (32, 28, 28)
        self.v1_v2_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)                            # (32, 28, 28) -> (32, 14, 14)

        self.v1_v2_conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) # (32, 14, 14) -> (64, 14, 14)
        self.v1_v2_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)                            # (64, 14, 14) -> (64, 7, 7)
        
        # --- 2. Pathway Split and Processing ---
        
        # 2a. Ventral Pathway (The 'What' Pathway - hOc4d, hOc5)
        # Characterized by more layers/filters for hierarchical feature extraction.
        self.ventral_conv = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1) # (64, 7, 7) -> (128, 7, 7)
        self.ventral_pool = nn.MaxPool2d(kernel_size=2, stride=2)                              # (128, 7, 7) -> (128, 3, 3)
        
        # Flattened size: 128 * 3 * 3 = 1152
        self.ventral_fc = nn.Linear(128 * 3 * 3, 256)
        
        # 2b. Dorsal Pathway (The 'Where/How' Pathway - hOc3d)
        # Characterized as being potentially shallower and focused on spatial/motion information.
        self.dorsal_conv = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)  # (64, 7, 7) -> (64, 7, 7)
        self.dorsal_pool = nn.MaxPool2d(kernel_size=3, stride=3)                                # (64, 7, 7) -> (64, 2, 2)
        
        # Flattened size: 64 * 2 * 2 = 256
        self.dorsal_fc = nn.Linear(64 * 2 * 2, 128)
        
        # --- 3. Final Classification Layer ---
        # Total concatenated features: 256 (Ventral) + 128 (Dorsal) = 384
        self.classifier = nn.Linear(256 + 128, 10) # 10 classes for Fashion MNIST

    def forward(self, x):
        
        # 1. Initial Feature Extraction (Shared V1/V2)
        x = self.v1_v2_pool1(F.relu(self.v1_v2_conv1(x)))
        shared_features = self.v1_v2_pool2(F.relu(self.v1_v2_conv2(x)))
        
        # 2. Pathway Split
        # 2a. Ventral Pathway
        v = self.ventral_pool(F.relu(self.ventral_conv(shared_features)))
        v = v.view(v.size(0), -1) # Flatten
        v_out = F.relu(self.ventral_fc(v))
        
        # 2b. Dorsal Pathway
        d = self.dorsal_pool(F.relu(self.dorsal_conv(shared_features)))
        d = d.view(d.size(0), -1) # Flatten
        d_out = F.relu(self.dorsal_fc(d))
        
        # 3. Concatenation and Classification
        combined = torch.cat([v_out, d_out], dim=1)
        output = self.classifier(combined)
        
        # The feature dictionary is needed later for RSA (Step 6).
        return output, {'v1_v2': shared_features, 'ventral_fc': v_out, 'dorsal_fc': d_out}

# --- Example of creating an instance (for testing/setup) ---
# model = ReferenceVisualNetwork()
# dummy_input = torch.randn(64, 1, 28, 28) # Batch size of 64
# output = model(dummy_input)
# print(f"Output shape: {output.shape}") # Should be (64, 10)

In [2]:
file_paths = {
    'V1_left': 'Human_brain_data/Area hOc1 (V1, 17, CalcS) left - cell density/tabular.csv',
    'V1_right': 'Human_brain_data/Area hOc1 (V1, 17, CalcS) right - cell density/tabular.csv',
    'V2_right': 'Human_brain_data/Area hOc2 (V2, 18) right - cell density/tabular.csv',
    'hOc3d_left': 'Human_brain_data/Area hOc3d (Cuneus) left - cell density/tabular.csv',
    'hOc4d_right': 'Human_brain_data/Area hOc4d (Cuneus) right - cell density/tabular.csv',
    'hOc5_right': 'Human_brain_data/Area hOc5 (LOC) right - cell density/tabular.csv',
}

# The column name from your readme file
DENSITY_COL = 'Segmented cell body density (detected cells / 0.1mm3)'

# Base filter count for the least dense region (F_base)
BASE_FILTER_COUNT = 64
# Scaling factor for shared layers (V1_conv1 is half the size of V1_conv2)
SHARED_CONV1_RATIO = 0.5


#CALCULATE AVERAGE DENSITIES

avg_densities = {}
all_densities = []

for region, path in file_paths.items():
    if os.path.exists(path):
        try:
            df = pd.read_csv(path)
            # Calculate the mean density across the 100 measurements
            mean_density = df[DENSITY_COL].mean()
            avg_densities[region] = mean_density
            all_densities.append(mean_density)
        except Exception as e:
            print(f"Error reading {path}: {e}")
            avg_densities[region] = np.nan
    else:
        print(f"File not found for {region}. Please check path: {path}")
        avg_densities[region] = np.nan
        
# Filter out NaNs if files were missing
valid_densities = [d for d in all_densities if not np.isnan(d)]

if not valid_densities:
    raise ValueError("No valid density data was loaded. Please fix file paths.")

# Find the minimum density (D_min) for scaling
MIN_DENSITY = min(valid_densities)

print(f"Average Densities (cells/0.1mm3): {avg_densities}")
print(f"Minimum Density (D_min): {MIN_DENSITY:.2f}")


#MAP DENSITY TO ARCHITECTURAL COMPONENTS\

# 1. Shared V1/V2 (Averaging V1 left, V1 right, V2 right)
shared_v1_v2_avg_density = np.mean([
    avg_densities['V1_left'], avg_densities['V1_right'], avg_densities['V2_right']
])
# 2. Dorsal Pathway (hOc3d left)
dorsal_avg_density = avg_densities['hOc3d_left']
# 3. Ventral Pathway (Averaging hOc4d right, hOc5 right)
ventral_avg_density = np.mean([
    avg_densities['hOc4d_right'], avg_densities['hOc5_right']
])

# Calculate scaling factors
scaling_factors = {
    'Shared_V1_V2': shared_v1_v2_avg_density / MIN_DENSITY,
    'Dorsal': dorsal_avg_density / MIN_DENSITY,
    'Ventral': ventral_avg_density / MIN_DENSITY,
}

#\CALCULATE FINAL FILTER COUNTS\

# Calculate the raw size for V1_V2_CONV2_CHANNELS before rounding
conv2_size_raw = scaling_factors['Shared_V1_V2'] * BASE_FILTER_COUNT

DENSITY_SCALED_FILTERS = {
    # Shared V1/V2 Layers
    # Uses the raw calculated size
    'V1_V2_CONV2_CHANNELS': int(np.round(conv2_size_raw)),
    # Uses the raw calculated size multiplied by 0.5 ratio
    'V1_V2_CONV1_CHANNELS': int(np.round(conv2_size_raw * SHARED_CONV1_RATIO)),
    
    # Dorsal Pathway
    'DORSAL_CONV_CHANNELS': int(np.round(scaling_factors['Dorsal'] * BASE_FILTER_COUNT)),
    'DORSAL_FC_SIZE': int(np.round(scaling_factors['Dorsal'] * (BASE_FILTER_COUNT * 2))), 
    
    # Ventral Pathway
    'VENTRAL_CONV_CHANNELS': int(np.round(scaling_factors['Ventral'] * BASE_FILTER_COUNT)),
    'VENTRAL_FC_SIZE': int(np.round(scaling_factors['Ventral'] * (BASE_FILTER_COUNT * 4))),
}

# Ensure all channel counts are at least 1 and round to nearest power of 2 for CNN efficiency
def round_to_power_of_2(n):
    if n <= 0: return 2
    return int(2**np.round(np.log2(n)))

for key in DENSITY_SCALED_FILTERS:
    if 'CHANNELS' in key:
        # Round convolutional channel counts to powers of 2 (32, 64, 128, etc.)
        DENSITY_SCALED_FILTERS[key] = round_to_power_of_2(DENSITY_SCALED_FILTERS[key])
    elif 'SIZE' in key:
        # Keep FC sizes as general integers, rounding to nearest 32
        DENSITY_SCALED_FILTERS[key] = max(32, int(np.round(DENSITY_SCALED_FILTERS[key] / 32) * 32))

# Recalculate V1_V2_CONV1 based on the FINAL, rounded V1_V2_CONV2 value
DENSITY_SCALED_FILTERS['V1_V2_CONV1_CHANNELS'] = round_to_power_of_2(
    DENSITY_SCALED_FILTERS['V1_V2_CONV2_CHANNELS'] // 2
)


print("\n--- FINAL SCALED FILTER COUNTS FOR CONSTRAINED MODEL ---")
print(DENSITY_SCALED_FILTERS)
print("----------------------------------------------------------")

Average Densities (cells/0.1mm3): {'V1_left': 90.42985758514597, 'V1_right': 90.42985758514597, 'V2_right': 76.51516244077028, 'hOc3d_left': 72.07978610755333, 'hOc4d_right': 61.15705207380484, 'hOc5_right': 63.61690940972587}
Minimum Density (D_min): 61.16

--- FINAL SCALED FILTER COUNTS FOR CONSTRAINED MODEL ---
{'V1_V2_CONV2_CHANNELS': 64, 'V1_V2_CONV1_CHANNELS': 32, 'DORSAL_CONV_CHANNELS': 64, 'DORSAL_FC_SIZE': 160, 'VENTRAL_CONV_CHANNELS': 64, 'VENTRAL_FC_SIZE': 256}
----------------------------------------------------------


In [3]:
V1_CONV1_C = 32    # V1/V2 CONV1 channels
V1_CONV2_C = 64    # V1/V2 CONV2 channels

D_CONV_C = 64       # Dorsal Conv channels (hOc3d)
D_FC_S = 160       # Dorsal FC size

V_CONV_C = 64     # Ventral Conv channels (hOc4d/hOc5)
V_FC_S = 256      # Ventral FC size

#SECOND VARIANT OF THE HUMAN VISUAL SYSTEM
class ConstrainedVisualNetwork(nn.Module):
    """
    A network constrained by human neuroanatomical data (cell density and SC).
    """
    def __init__(self):
        super(ConstrainedVisualNetwork, self).__init__()
        
        # --- 1. Shared V1/V2 (Size constrained by V1/V2 density) ---
        self.v1_v2_conv1 = nn.Conv2d(1, V1_CONV1_C, kernel_size=3, padding=1)
        self.v1_v2_pool1 = nn.MaxPool2d(2, 2)

        self.v1_v2_conv2 = nn.Conv2d(V1_CONV1_C, V1_CONV2_C, kernel_size=3, padding=1)
        self.v1_v2_pool2 = nn.MaxPool2d(2, 2) # Output size: (V1_CONV2_C, 7, 7)
        
        # --- 2. Ventral Pathway (hOc4d, hOc5 - Size constrained by density) ---
        self.ventral_conv = nn.Conv2d(V1_CONV2_C, V_CONV_C, kernel_size=3, padding=1)
        self.ventral_pool = nn.MaxPool2d(2, 2)
        
        # Flattened size: V_CONV_C * 3 * 3
        VENTRAL_FLATTEN_SIZE = V_CONV_C * 3 * 3
        self.ventral_fc = nn.Linear(VENTRAL_FLATTEN_SIZE, V_FC_S)
        
        # --- 3. Dorsal Pathway (hOc3d - Size constrained by density) ---
        self.dorsal_conv = nn.Conv2d(V1_CONV2_C, D_CONV_C, kernel_size=3, padding=1)
        self.dorsal_pool = nn.MaxPool2d(3, 3)
        
        # Flattened size: D_CONV_C * 2 * 2
        DORSAL_FLATTEN_SIZE = D_CONV_C * 2 * 2
        self.dorsal_fc = nn.Linear(DORSAL_FLATTEN_SIZE, D_FC_S)
        
        # --- 4. Structural Connectivity Module (SC) ---
        # This layer transforms V1_CONV1_C features (from V1_conv1 output) to match 
        # the spatial/channel dimensions of the Dorsal input (V1_CONV2_C, 7, 7).
        # Adjust the input/output channels and stride based on your SC data (e.g., V1->hOc3d).
        self.v1_to_dorsal_skip = nn.Conv2d(V1_CONV1_C, V1_CONV2_C, kernel_size=1, stride=4, bias=False) 
        
        # 5. Final Classifier
        self.classifier = nn.Linear(V_FC_S + D_FC_S, 10) 
        
        self._initialize_weights()

    def _initialize_weights(self):
        # Initializes weights for stability, matching the template provided earlier
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, 0, 0.01)
                init.constant_(m.bias, 0)

    def forward(self, x):
        
        # 1. Initial V1 processing (Source for SC skip connection)
        v1_features = F.relu(self.v1_v2_conv1(x)) # (B, V1_CONV1_C, 28, 28)
        x = self.v1_v2_pool1(v1_features)         # (B, V1_CONV1_C, 14, 14)
        
        # 2. V2 Processing (Shared features)
        shared_features = self.v1_v2_pool2(F.relu(self.v1_v2_conv2(x))) # (B, V1_CONV2_C, 7, 7)
        
        # 3. Ventral Pathway
        v = self.ventral_pool(F.relu(self.ventral_conv(shared_features)))
        ventral_conv_out = v 
        
        # --- CRITICAL: STRUCTURAL CONNECTIVITY (SC) IMPLEMENTATION ---
        
        # Prepare SC skip: Transforms V1 features (V1_CONV1_C, 28, 28) 
        # to match the shape of the dorsal input (V1_CONV2_C, 7, 7).
        v1_skip = self.v1_to_dorsal_skip(v1_features) 

        # 4. Dorsal Pathway with SC Constraint
        # Input to dorsal pathway = Shared V2 features + Direct V1 Skip (Residual connection)
        # 
        # !!! CHECK YOUR CONNECTIVITY DATA AND ADJUST THE ADDITION BELOW !!!
        # If your data shows a strong V1 -> hOc3d connection, keep this:
        d_input = shared_features + v1_skip
        
        d = self.dorsal_pool(F.relu(self.dorsal_conv(d_input))) 
        
        # Flatten and FC for both pathways
        v_fc_in = ventral_conv_out.view(ventral_conv_out.size(0), -1)
        v_out = F.relu(self.ventral_fc(v_fc_in))
        
        d_fc_in = d.view(d.size(0), -1)
        d_out = F.relu(self.dorsal_fc(d_fc_in))
        
        # 5. Concatenation and Classification
        combined = torch.cat([v_out, d_out], dim=1)
        output = self.classifier(combined)
        
        return output, {'v1_v2': shared_features, 'ventral_fc': v_out, 'dorsal_fc': d_out}

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import time

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
LEARNING_RATE = 0.001
N_EPOCHS = 10 

def load_fashion_mnist():
    """Loads and prepares the Fashion MNIST dataset."""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Download and load training dataset
    train_dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=True, download=True, transform=transform
    )

    # Split training set into training and validation
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_data, val_data = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
    
    # Download and load test dataset
    test_dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=False, download=True, transform=transform
    )
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, val_loader, test_loader

def train_model(model, train_loader, val_loader, epochs=N_EPOCHS, lr=LEARNING_RATE, device=DEVICE):
    """
    Trains the given model and tracks loss/accuracy.
    """
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    
    print(f"Starting training for {type(model).__name__} on {device}...")
    start_time = time.time()
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            # The forward pass now returns output AND features
            outputs, _ = model(inputs) 
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        avg_train_loss = running_loss / len(train_loader)
        
        # Validation
        val_loss, val_acc = evaluate_model(model, val_loader, criterion, device)
        
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    end_time = time.time()
    print(f"Training finished in {(end_time - start_time):.2f} seconds.")
    return model, history

def evaluate_model(model, data_loader, criterion, device):
    """Evaluates the model on the given data loader."""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs, _ = model(inputs)
            
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = correct / total
    return avg_loss, accuracy

print("Training utility functions defined.")

Training utility functions defined.


In [5]:

# 1. Setup and Data Loading
train_loader, val_loader, test_loader = load_fashion_mnist() # This function was provided earlier

# 2. Train the Reference Model
print("--- Training Reference Network ---")
ref_model = ReferenceVisualNetwork()
ref_model_trained, ref_history = train_model(ref_model, train_loader, val_loader)
torch.save(ref_model_trained.state_dict(), 'reference_model.pth') # Save the model

# 3. Train the Constrained Model
print("\n--- Training Constrained Network ---")
constrained_model = ConstrainedVisualNetwork() 
constrained_model_trained, const_history = train_model(constrained_model, train_loader, val_loader)
torch.save(constrained_model_trained.state_dict(), 'constrained_model.pth') # Save the model

print("\nModels trained and saved. Ready for Step 6: RSA.")

100%|██████████| 26.4M/26.4M [00:13<00:00, 1.94MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 129kB/s]
100%|██████████| 4.42M/4.42M [00:02<00:00, 1.88MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 26.0kB/s]


--- Training Reference Network ---
Starting training for ReferenceVisualNetwork on cpu...
Epoch 1/10: Train Loss: 0.5544 | Val Loss: 0.3608 | Val Acc: 0.8667
Epoch 2/10: Train Loss: 0.3196 | Val Loss: 0.2901 | Val Acc: 0.8900
Epoch 3/10: Train Loss: 0.2701 | Val Loss: 0.2483 | Val Acc: 0.9073
Epoch 4/10: Train Loss: 0.2365 | Val Loss: 0.2590 | Val Acc: 0.9025
Epoch 5/10: Train Loss: 0.2118 | Val Loss: 0.2461 | Val Acc: 0.9068
Epoch 6/10: Train Loss: 0.1903 | Val Loss: 0.2494 | Val Acc: 0.9097
Epoch 7/10: Train Loss: 0.1697 | Val Loss: 0.2248 | Val Acc: 0.9168
Epoch 8/10: Train Loss: 0.1521 | Val Loss: 0.2243 | Val Acc: 0.9177
Epoch 9/10: Train Loss: 0.1377 | Val Loss: 0.2467 | Val Acc: 0.9115
Epoch 10/10: Train Loss: 0.1213 | Val Loss: 0.2294 | Val Acc: 0.9201
Training finished in 306.83 seconds.

--- Training Constrained Network ---
Starting training for ConstrainedVisualNetwork on cpu...
Epoch 1/10: Train Loss: 0.6160 | Val Loss: 0.3845 | Val Acc: 0.8609
Epoch 2/10: Train Loss: 0.348

### Exercise 7 — Representational Similarity Analysis (RSA)


In [8]:
torch.set_grad_enabled(False)

def _get_subset(loader, n=1000):
    xs, ys, c = [], [], 0
    for x, y in loader:
        xs.append(x); ys.append(y); c += x.size(0)
        if c >= n: break
    X = torch.cat(xs, dim=0)[:n]
    Y = torch.cat(ys, dim=0)[:n]
    return X, Y

def collect_activations(model, data_tensor, device):
    model.eval().to(device)
    feats = {"v1_v2": [], "ventral_fc": [], "dorsal_fc": []}
    bs = 128
    for i in range(0, data_tensor.size(0), bs):
        xb = data_tensor[i:i+bs].to(device)
        _, f = model(xb)
        for k in feats:
            v = f[k].detach().cpu().numpy()
            feats[k].append(v)
    for k in feats:
        feats[k] = np.concatenate(feats[k], axis=0)
    return feats

def rdm_vector(X):
    Xc = X - X.mean(0, keepdims=True)
    cov = Xc @ Xc.T
    v = np.sqrt(np.clip(np.diag(cov), 1e-12, None))
    corr = cov / (v[:, None] * v[None, :])
    D = 1.0 - corr
    iu = np.triu_indices_from(D, k=1)
    return D[iu]

def rsa_between(A, B):
    a = rdm_vector(A)
    b = rdm_vector(B)
    s, _ = spearmanr(a, b)
    return float(s)


In [10]:
def _to_2d(X):
    X = np.asarray(X)
    if X.ndim == 1:
        X = X[None, :]
    elif X.ndim > 2:
        X = X.reshape(X.shape[0], -1)
    return X

def rdm_vector(X):
    X = _to_2d(X).astype(np.float64, copy=False)
    Xc = X - X.mean(0, keepdims=True)
    cov = Xc @ Xc.T
    v = np.sqrt(np.clip(np.diag(cov), 1e-12, None))
    corr = cov / (v[:, None] * v[None, :])
    D = 1.0 - corr
    iu = np.triu_indices_from(D, 1)
    return D[iu]


In [11]:
device = DEVICE
X_test, y_test = _get_subset(test_loader, n=1000)

ref_model_trained.eval().to(device)
constrained_model_trained.eval().to(device)

ref_feats = collect_activations(ref_model_trained, X_test, device)
con_feats = collect_activations(constrained_model_trained, X_test, device)

layers = ["v1_v2", "ventral_fc", "dorsal_fc"]
rsa_scores = {l: rsa_between(ref_feats[l], con_feats[l]) for l in layers}

print("RSA (Spearman) — Reference vs Constrained")
for l in layers:
    print(l, round(rsa_scores[l], 4))

np.savez("rsa_scores_ref_vs_constrained.npz", **rsa_scores)


RSA (Spearman) — Reference vs Constrained
v1_v2 0.9869
ventral_fc 0.8335
dorsal_fc 0.5202


### Exercise 8 — Functional connectivity before vs after training



In [12]:
def layerwise_fc(act_dict):
    keys = ["v1_v2", "ventral_fc", "dorsal_fc"]
    S = []
    for k in keys:
        X = act_dict[k].reshape(act_dict[k].shape[0], -1).mean(axis=1)
        S.append(X)
    S = np.stack(S, axis=0)
    S = S - S.mean(1, keepdims=True)
    C = np.corrcoef(S)
    return keys, C

def save_matrix(mat, title, path):
    plt.figure()
    plt.imshow(mat, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()

X_small, y_small = _get_subset(test_loader, n=600)

ref_untrained = ReferenceVisualNetwork().to(DEVICE).eval()
con_untrained = ConstrainedVisualNetwork().to(DEVICE).eval()

ref_feats_before = collect_activations(ref_untrained, X_small, DEVICE)
ref_feats_after  = collect_activations(ref_model_trained, X_small, DEVICE)
con_feats_before = collect_activations(con_untrained, X_small, DEVICE)
con_feats_after  = collect_activations(constrained_model_trained, X_small, DEVICE)

_, ref_C_before = layerwise_fc(ref_feats_before)
_, ref_C_after  = layerwise_fc(ref_feats_after)
_, con_C_before = layerwise_fc(con_feats_before)
_, con_C_after  = layerwise_fc(con_feats_after)

save_matrix(ref_C_before, "Reference FC Before", "ref_fc_before.png")
save_matrix(ref_C_after,  "Reference FC After",  "ref_fc_after.png")
save_matrix(con_C_before, "Constrained FC Before", "con_fc_before.png")
save_matrix(con_C_after,  "Constrained FC After",  "con_fc_after.png")

def tri_upper(m):
    iu = np.triu_indices_from(m, 1)
    return m[iu]

print("ΔFC mean |After−Before|")
print("Reference:", float(np.mean(np.abs(tri_upper(ref_C_after) - tri_upper(ref_C_before)))))
print("Constrained:", float(np.mean(np.abs(tri_upper(con_C_after) - tri_upper(con_C_before)))))

np.savez("fc_matrices.npz",
         ref_before=ref_C_before, ref_after=ref_C_after,
         con_before=con_C_before, con_after=con_C_after)


ΔFC mean |After−Before|
Reference: 0.7812622246663143
Constrained: 0.7674857501470256


### Exercise 9  — Empirical connectivity comparison


In [18]:
emp = load_empirical_strengths()

if len(emp) == 0:
    print("No empirical CSVs found.")
else:
    def pick_first(d, names):
        for n in names:
            df = d.get(n)
            if isinstance(df, pd.DataFrame) and not df.empty:
                return df
        return None

    def mean_conn(df, targets):
        if df is None or df.empty:
            return 0.0
        cands = [c for c in df.columns if "weight" in c.lower() or "strength" in c.lower()]
        if cands:
            wcol = cands[0]
        else:
            nums = df.select_dtypes(include=[np.number]).columns.tolist()
            if not nums:
                return 0.0
            wcol = nums[0]
        sdf = df.astype(str)
        mask = pd.Series(False, index=df.index)
        for t in targets:
            mask = mask | sdf.apply(lambda r: r.str.contains(t, case=False, regex=False)).any(axis=1)
        sel = df.loc[mask]
        if sel.empty:
            return 0.0
        vals = pd.to_numeric(sel[wcol], errors="coerce").dropna().abs()
        if vals.empty:
            return 0.0
        return float(vals.mean())

    V1_df      = pick_first(emp, ["V1_left", "V1_right"])
    Ventral_df = pick_first(emp, ["hOc4d_right", "hOc5_right"])
    Dorsal_df  = pick_first(emp, ["hOc3d_left"])

    M = np.eye(3)
    M[0,1] = M[1,0] = mean_conn(V1_df, ["hOc4", "LOC", "hOc5"])
    M[0,2] = M[2,0] = mean_conn(V1_df, ["hOc3"])
    M[1,2] = M[2,1] = mean_conn(Ventral_df, ["hOc3"])

    save_matrix(M, "Empirical 3-node FC (proxy)", "empirical_fc_proxy.png")

    f = np.load("fc_matrices.npz")
    ref_fc_after = f["ref_after"]
    con_fc_after = f["con_after"]

    def tri(m):
        iu = np.triu_indices_from(m, 1)
        return m[iu]

    er = spearmanr(tri(M), tri(ref_fc_after))[0]
    ec = spearmanr(tri(M), tri(con_fc_after))[0]
    print("Spearman(Empirical, Reference After):", round(float(er), 4))
    print("Spearman(Empirical, Constrained After):", round(float(ec), 4))


Spearman(Empirical, Reference After): -0.5
Spearman(Empirical, Constrained After): -0.5


### Exercise 10 — t-SNE of representational space


In [19]:
def tsne_plot(feats, labels, title, path):
    X = feats.reshape(feats.shape[0], -1)
    idx = np.random.permutation(len(labels))[:1000]
    Xs = X[idx]
    ys = labels.numpy()[idx]
    Z = TSNE(n_components=2, init="pca", learning_rate="auto", perplexity=30).fit_transform(Xs)
    plt.figure()
    for c in np.unique(ys):
        m = ys == c
        plt.scatter(Z[m,0], Z[m,1], s=8, label=str(c))
    plt.title(title)
    plt.legend(markerscale=2, fontsize=8)
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()

ref_pen = np.concatenate([ref_feats["ventral_fc"], ref_feats["dorsal_fc"]], axis=1)
con_pen = np.concatenate([con_feats["ventral_fc"], con_feats["dorsal_fc"]], axis=1)

tsne_plot(ref_pen, y_test, "t-SNE Reference", "tsne_reference.png")
tsne_plot(con_pen, y_test, "t-SNE Constrained", "tsne_constrained.png")
